In [5]:
!pip install -q transformers torch

import torch
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForQuestionAnswering,
    AutoModelForSeq2SeqLM,
    AutoModelForTokenClassification,
    # Added explicit imports for the last section
    BartTokenizer,
    BartForSequenceClassification,
)

print("Torch device available:", "cuda" if torch.cuda.is_available() else "cpu")

# ============================================================

# 1) SENTIMENT ANALYSIS (Finance sentence)

# ============================================================

print("\n================ 1) SENTIMENT ANALYSIS ================")

text = "The company's stock jumped 7% after the positive earnings report."

# ---- A) Pipeline version (easy) ----

sentiment_pipe = pipeline("sentiment-analysis")
print("Pipeline output:", sentiment_pipe(text))

# ---- B) Model + tokenizer version ----

sentiment_model_id = "distilbert-base-uncased-finetuned-sst-2-english"

sent_tok = AutoTokenizer.from_pretrained(sentiment_model_id)
sent_model = AutoModelForSequenceClassification.from_pretrained(sentiment_model_id)

inputs = sent_tok(text, return_tensors="pt")

with torch.no_grad():
    outputs = sent_model(**inputs)
    logits = outputs.logits

pred_id = torch.argmax(logits, dim=-1).item()
label = sent_model.config.id2label[pred_id]

print("Model+tokenizer label:", label)

# ============================================================

# 2) QUESTION ANSWERING (Finance context)

# ============================================================

print("\n================ 2) QUESTION ANSWERING ================")

context = (
"In Q4, Apple reported revenue of 100 billion dollars and a net profit of 25 billion dollars. "
"The strong iPhone sales contributed to this performance."
)
question = "What was Apple's net profit?"

# ---- A) Pipeline version ----

qa_pipe = pipeline("question-answering")
print("Pipeline output:", qa_pipe({"question": question, "context": context}))

# ---- B) Model + tokenizer version ----

qa_model_id = "distilbert-base-uncased-distilled-squad"

qa_tok = AutoTokenizer.from_pretrained(qa_model_id)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_id)

qa_inputs = qa_tok(question, context, return_tensors="pt")

with torch.no_grad():
    qa_outputs = qa_model(**qa_inputs)

start_idx = torch.argmax(qa_outputs.start_logits)
end_idx = torch.argmax(qa_outputs.end_logits)

answer_ids = qa_inputs["input_ids"][0][start_idx : end_idx + 1]
answer_text = qa_tok.decode(answer_ids)

print("Model+tokenizer answer:", answer_text)

# ============================================================

# 3) SUMMARIZATION (Finance news paragraph)

# ============================================================

print("\n================ 3) SUMMARIZATION ================")

finance_paragraph = (
"Microsoft released its quarterly results today. The company reported strong growth "
"in its cloud business Azure, while personal computing revenue was flat. "
"Overall, total revenue increased compared to the same quarter last year."
)

# ---- A) Pipeline version ----

summ_pipe = pipeline("summarization", model="facebook/bart-large-cnn")
print("Pipeline summary:", summ_pipe(finance_paragraph, max_length=40, min_length=10, do_sample=False))

# ---- B) Model + tokenizer version ----

summ_model_id = "facebook/bart-large-cnn"

summ_tok = AutoTokenizer.from_pretrained(summ_model_id)
summ_model = AutoModelForSeq2SeqLM.from_pretrained(summ_model_id)

summ_inputs = summ_tok(finance_paragraph, return_tensors="pt")

with torch.no_grad():
    summary_ids = summ_model.generate(
        **summ_inputs,
        max_new_tokens=40,
        num_beams=4,
        early_stopping=True,
    )

summary_text = summ_tok.decode(summary_ids[0], skip_special_tokens=True)
print("Model+tokenizer summary:", summary_text)

# ============================================================

# 4) TRANSLATION (Finance phrase EN -> DE)

# ============================================================

print("\n================ 4) TRANSLATION ================")

english_text = "The central bank decided to increase interest rates by 0.5%."

# ---- A) Pipeline version ----

trans_pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-en-de")
print("Pipeline translation:", trans_pipe(english_text))

# ---- B) Model + tokenizer version ----

trans_model_id = "Helsinki-NLP/opus-mt-en-de"

trans_tok = AutoTokenizer.from_pretrained(trans_model_id)
trans_model = AutoModelForSeq2SeqLM.from_pretrained(trans_model_id)

trans_inputs = trans_tok(english_text, return_tensors="pt")

with torch.no_grad():
    trans_ids = trans_model.generate(**trans_inputs, max_new_tokens=40)

german_text = trans_tok.decode(trans_ids[0], skip_special_tokens=True)
print("Model+tokenizer translation:", german_text)

# ============================================================

# 5) NER (Named Entity Recognition) on finance sentence

# ============================================================

print("\n================ 5) NER (ENTITY RECOGNITION) ================")

ner_text = "Goldman Sachs invested 500 million dollars in a new fintech startup."

# ---- A) Pipeline version ----

ner_pipe = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")
print("Pipeline NER:", ner_pipe(ner_text))

# ---- B) Model + tokenizer version ----

ner_model_id = "dslim/bert-base-NER"

ner_tok = AutoTokenizer.from_pretrained(ner_model_id)
ner_model = AutoModelForTokenClassification.from_pretrained(ner_model_id)

ner_inputs = ner_tok(ner_text, return_tensors="pt")

with torch.no_grad():
    ner_outputs = ner_model(**ner_inputs)

# Get predicted label id for each token

pred_ids = torch.argmax(ner_outputs.logits, dim=-1)[0]

print("Model+tokenizer NER tokens and labels:")
for token_id, label_id in zip(ner_inputs["input_ids"][0], pred_ids):
    token_str = ner_tok.decode([token_id]).strip()
    label_str = ner_model.config.id2label[label_id.item()]
    print(f"{token_str:15s} -> {label_str}")

# ============================================================

# 6) ZERO-SHOT CLASSIFICATION (finance topic)

# ============================================================

print("\n================ 6) ZERO-SHOT CLASSIFICATION ================")

zs_text = "Bitcoin fell 5% as investors moved money into government bonds."
candidate_labels = ["stocks", "crypto", "macroeconomy"]

# ---- A) Pipeline version ----

# This model is specifically for zero-shot classification via NLI (Natural Language Inference)
zs_pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
print("Pipeline zero-shot:", zs_pipe(zs_text, candidate_labels=candidate_labels))

# ---- B) Model + tokenizer version ----

# Removed the redundant re-import
zs_model_id = "facebook/bart-large-mnli"

zs_tok = AutoTokenizer.from_pretrained(zs_model_id)
zs_model = AutoModelForSequenceClassification.from_pretrained(zs_model_id)

def zero_shot_simple(text, labels):
    scores = []
    # Ensure the correct label map is used (ENTAILMENT is common for NLI-based zero-shot)
    entailment_id = zs_model.config.label2id["entailment"] # Changed 'ENTAILMENT' to 'entailment'
    for label in labels:
        # premise = text, hypothesis = "This text is about {label}."
        pair = zs_tok(text, f"This text is about {label}.", return_tensors="pt", truncation=True)
        with torch.no_grad():
            logits = zs_model(**pair).logits
            # For MNLI: labels are ['CONTRADICTION', 'NEUTRAL', 'ENTAILMENT']
            probs = torch.softmax(logits, dim=-1)[0]
            entailment_score = probs[entailment_id].item()
            scores.append(entailment_score)
    return scores

zs_scores = zero_shot_simple(zs_text, candidate_labels)
print("Model+tokenizer zero-shot scores:")
for label, score in zip(candidate_labels, zs_scores):
    print(f"{label:12s}: {score:.3f}")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Torch device available: cuda

================ 1) SENTIMENT ANALYSIS ================


Device set to use cuda:0


Pipeline output: [{'label': 'POSITIVE', 'score': 0.724334180355072}]


No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


...

Pipeline NER: [{'entity_group': 'ORG', 'score': np.float32(0.99910915), 'word': 'Goldman Sachs', 'start': 0, 'end': 13}]


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model+tokenizer NER tokens and labels:
[CLS]           -> O
Goldman         -> B-ORG
Sachs           -> I-ORG
invested        -> O
500             -> O
million         -> O
dollars         -> O
in              -> O
a               -> O
new             -> O
fin             -> O
##tech          -> O
start           -> O
##up            -> O
.               -> O
[SEP]           -> O

================ 6) ZERO-SHOT CLASSIFICATION ================


Device set to use cuda:0


Pipeline zero-shot: {'sequence': 'Bitcoin fell 5% as investors moved money into government bonds.', 'labels': ['crypto', 'macroeconomy', 'stocks'], 'scores': [0.9434500336647034, 0.04950190335512161, 0.007048130501061678]}
Model+tokenizer zero-shot scores:
stocks      : 0.000
crypto      : 0.325
macroeconomy: 0.047
